In [1]:
# Lesson 1: Simple ReAct Agent from Scratch
import openai
import re
import httpx
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup

_ = load_dotenv()
from openai import OpenAI
client = OpenAI()

chat_completion = client.chat.completions.create(
    model="mistral-nemo",
    messages=[{"role": "user", "content": "Hello world"}]
)
print(chat_completion.choices[0].message.content)

class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="mistral-nemo", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
        
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

scrape_page:
e.g. scrape_page: https://example.com
Scrapes the content of the provided URL and returns the main text content.

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dog's weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weighs 51 lbs

You then output:

Answer: A bulldog weighs 51 lbs
""".strip()

def calculate(what):
    try:
        return str(eval(what))
    except Exception as e:
        return f"Error in calculation: {e}"

def average_dog_weight(name):
    weights = {
        "Scottish Terrier": "20 lbs",
        "Border Collie": "37 lbs",
        "Toy Poodle": "7 lbs"
    }
    return f"{name} average weight is {weights.get(name, '50 lbs')}"

def scrape_page(url):
    try:
        response = httpx.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text_content = '\n'.join([para.get_text() for para in paragraphs])
        return text_content if text_content else "No textual content found on the page."
    except httpx.RequestError as e:
        return f"An error occurred while requesting the page: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight,
    "scrape_page": scrape_page
}

abot = Agent(prompt)
result = abot("How much does a toy poodle weigh?")
print(result)

result = average_dog_weight("Toy Poodle")
print(result)

next_prompt = "Observation: {}".format(result)
abot(next_prompt)
print(abot.messages)

abot = Agent(prompt)
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)
abot(next_prompt)

next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)
abot(next_prompt)

next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)
abot(next_prompt)

### Add loop 
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to select action
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return
            
# Example query using the new scrape_page action
# question = "Can you provide the main content of OpenAI's homepage?"
# query(question)


<>:138: SyntaxWarning: invalid escape sequence '\w'
<>:138: SyntaxWarning: invalid escape sequence '\w'
/var/folders/82/vkdwm96j5f1f23w2mf5z6n280000gn/T/ipykernel_10271/2187654334.py:138: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to select action


Hello! How are you doing today? Let me know if there's anything I can help with. 😊
Thought: I should look up the average weight of a Toy Poodle using average_dog_weight.
Action: average_dog_weight: Toy Poodle
PAUSE
Toy Poodle average weight is 7 lbs
[{'role': 'system', 'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nscrape_page:\ne.g. scrape_page: https://example.com\nScrapes the content of the provided URL and returns th

In [2]:
def test_scrape():
    test_url = "https://www.geeksforgeeks.org/performing-google-search-using-python-code/"  # Replace with a valid URL you'd like to scrape
    content = scrape_page(test_url)
    print("Scraped Content:\n", content)

test_scrape()


Scraped Content:
 Notifications
Let’s say you are working on a project that needs to do web scraping but you don’t know websites on which scraping is to be performed beforehand instead you are required to perform a google search and then proceed according to google search results to a few websites. In that case, you need google search results for your different queries.
Installation google package has one dependency on beautifulsoup which needs to be installed first.  
Then install the google package  
Required Function and its parameters 
Python codes on how to do a google search using python script
Example1: google_search.py  
 

 
 



Output: 

Let’s perform a google search manually and verify our result

 

Example 2: google_search.py 
 

 
 



Output: 

Let’s perform a google search manually and verify our result

Reference: Google python package
 
P

